# Import statements

In [1]:
from ultralytics import YOLO
import os
import shutil
import pandas as pd
from PIL import Image, ImageFilter
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.filters as skif
from skimage.io import imshow, imread
from skimage.color import rgb2yuv, rgb2hsv, rgb2gray, yuv2rgb, hsv2rgb
from scipy.signal import convolve2d
import argparse as args
import glob
from skimage.filters import gaussian
from skimage import img_as_ubyte
import requests

# Paths

In [6]:
model_path = r'C:/Users/bengi/Downloads/best_preprocessed.pt'
source_path = r'C:/Users/bengi/Downloads/Ankle Photos/healthy'
temp1_path = r'temp1'
temp2_path = r'temp2'
destination_path = r'C:/Users/bengi/Downloads/Ankle Photos/healthyV2'

# Functions

In [3]:
def mask(img_path):
    model = YOLO(model_path)
    
    results = model(img_path)
    
    selected_classes = [0, 1, 2, 3]
    
    image = cv2.imread(img_path)
    
    # Test if image worked
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    
    # Create the mask the size of the image with 0s everywhere
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    
    # Loop through the array of boxes for each box
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    
    # Create a mask of the same size as the image, with zeros everywhere    
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    for det in results[0].boxes:
        # Define the coordinates of the rectangle (top-left and bottom-right corners)
        for c in selected_classes:
            if (det.cls == c):
                xmin, ymin, xmax, ymax = det.xyxy[0]
                x1 = int(xmin)
                x2 = int(xmax)
                y1 = int(ymin)
                y2 = int(ymax)
            
                # Fill the region of interest in the mask with ones
                mask[y1:y2, x1:x2] = 1
                #num_maksed_pixels += (y2-y1) * (x2 - x1)
                # Apply the mask to the image
                masked_image = image_rgb * mask[:, :, np.newaxis]

    return masked_image

In [4]:
#==============================================================================
class ImageProcessor:
    def __init__(self, input_folder, output_folder):
        """Initialize the class"""
        self.input_folder = input_folder
        self.output_folder = output_folder
#------------------------------------------------------------------------------
    def SmoothImage(self,img):
        """Smoothing image"""
        smoothed_image = img_as_ubyte(gaussian(img, sigma=3, mode='constant', cval=0.0))
        return smoothed_image
#------------------------------------------------------------------------------
    def SharpenImage(self, img):
        """Sharpen given image"""

        #TODO: Consider this procedure for sharpening instead:
        #https://scipy-lectures.org/advanced/image_processing/auto_examples/plot_sharpen.html

        kernel = np.array([[-1, -1, -1],
                           [-1,  9, -1],
                           [-1, -1, -1]])
        sharpened_image = cv2.filter2D(img, -1, kernel)
        #blurred = cv2.GaussianBlur(img, (9, 9), 10.0)
        #sharpened = cv2.addWeighted(img, 1.5, blurred, -0.5, 0)
        return sharpened_image
#------------------------------------------------------------------------------
    def InvertImage(self, img):
        """Inverting image"""
        q = np.average(img)
        avg = q / 255
        if avg < 0.5:
            x1 = np.array([25,255,10], np.uint8) #for example
            x2 = ~x1
            #print (x2)
            #image = cv2.imread('test_Rankle.png', 0)
            inverted_image = np.invert(img)
            return inverted_image
        else:
            return img
#------------------------------------------------------------------------------
    def SizeImage(self, img):
        """Sizing image"""
        #image = Image.open(img)
        sized_image = img.resize((500, 500))
        return sized_image
#------------------------------------------------------------------------------
    def CleanImage (self, img):
        """Removing background"""
        cleaned_image = remove(img)
        return cleaned_image
#------------------------------------------------------------------------------
    def ContrastImage (self, img):
        """Increasing contrast"""
        alpha = 1.5 # Contrast control
        beta = 10 # Brightness control
        contrasted_image = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        return contrasted_image
#------------------------------------------------------------------------------
    def EdgeDetection (self, img):
        """Detecting edges"""
        img_blur = cv2.GaussianBlur(img, (3,3), 0)
        sobel_y = cv2.Sobel(src=img, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
        m = 3
        n = 5
        edged_image = cv2.blur(sobel_y, (m,n))
        return edged_image
#------------------------------------------------------------------------------
    def Transform(self,img):
        """Tranform given image"""
        #smoothed_image = self.SmoothImage(img)
        sharpened_image = self.SharpenImage(img)
        inverted_image = self.InvertImage(sharpened_image)
        #sized_image = self.SizeImage(inverted_image)
        #cleaned_image = self.CleanImage(inverted_image)
        #contrasted_image = self.ContrastImage(inverted_image)
        #transformed_image = self.EdgeDetection(inverted_image)

        return inverted_image
#------------------------------------------------------------------------------
    def Run(self):
        """Scan files"""
        root = os.path.dirname(os.path.abspath("__file__"))
        input_folder = os.path.join(root, self.input_folder)
        output_folder = os.path.join(root, self.output_folder)

        for name in os.listdir(input_folder):
            input_file_path = os.path.join(input_folder, name)
            print("Processing file: {}".format(input_file_path))
            img = cv2.imread(input_file_path, 0)
            processed_img = self.Transform(img)

            #Save processed image
            output_name = "edged_image_{}".format(name)
            output_file_path = os.path.join(output_folder, output_name)

            print("Saving file: {}".format(output_file_path))
            if not cv2.imwrite(output_file_path, processed_img):
                print("Error: cannot save:{}".format(output_file_path))

#==============================================================================
def main(input_folder, output_folder):
    ip = ImageProcessor(input_folder, output_folder)
    ip.Run()
#==============================================================================

# Run

In [ ]:
# Create the output folder if it doesn't exist
os.makedirs(destination_path, exist_ok=True)
os.makedirs(temp1_path, exist_ok=True)
os.makedirs(temp2_path, exist_ok=True)

input_folder =  source_path
output_folder = temp1_path
main(input_folder, output_folder)

# Iterate over all files in the input folder

for filename in os.listdir(temp1_path):
    # Construct full file path
    input_path = os.path.join(temp1_path, filename)

    # Ensure we are working with an image file
    if os.path.isfile(input_path):
        try:
            # Open the image file
            with Image.open(input_path) as img:
                # Convert the image to a NumPy array
                img_array = np.array(img)

                #new_image = preprocess(input_path)
                
                masked_image = mask(input_path)

                # Construct the output path
                output_path = os.path.join(temp2_path, filename)

                # Save the filtered image
                plt.imsave(output_path, masked_image)
                print(f"Processed and saved: {output_path}")
        except Exception as e:
            print(f"Failed to process {input_path}: {e}")
print("Process complete")

# Delete temp folders

In [ ]:
shutil.rmtree(temp1_path)
print(f"Deleted folder and all its contents: {temp1_path}")

In [ ]:
shutil.rmtree(temp2_path)
print(f"Deleted folder and all its contents: {temp2_path}")